In [2]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
import pickle
import warnings
warnings.filterwarnings(action='once')


with open('models/tokenizer_magyar.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

models = {"Petőfi Sándor": tf.keras.models.load_model('models/petofi_összes.mdl_wts.hdf5'),
          "Babits Mihály": tf.keras.models.load_model('models/babits_összes.mdl_wts.hdf5'),
          "Karinthy Frigyes": tf.keras.models.load_model('models/karinthy_összes.mdl_wts.hdf5')}

    
tf.random.set_seed(42)

def complete_text(text, model_name, n_chars=50, temperature=1):
    model = models[model_name]
    def preprocess(texts):
        X = np.array(tokenizer.texts_to_sequences(texts)) - 1
        return tf.one_hot(X, 52)

    def next_char(text, temperature=1):
        X_new = preprocess([text])
        y_proba = model.predict(X_new)[0, -1:, :]
        rescaled_logits = tf.math.log(y_proba) / temperature
        char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
        return tokenizer.sequences_to_texts(char_id.numpy())[0]
    
    
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [9]:
a = complete_text("Zsuzsanna ",n_chars=100 ,temperature=0.5, model_name="Petőfi Sándor")
b = complete_text("Zsuzsanna ",n_chars=100 ,temperature=0.5, model_name="Babits Mihály")
c = complete_text("Zsuzsanna ",n_chars=100 ,temperature=0.5, model_name="Karinthy Frigyes")

In [10]:
a,b,c

('Zsuzsanna fekete.hogy a királyi szerelmem szegény ellennézem nem lesz a nap itt egy istent lett az oldalásba, ',
 'Zsuzsanna alig egyicsotász, kivigyegettfested, ilathond a falunk mind nem tudja,kincs felhőtet egényet a hajda',
 'Zsuzsanna volt a szép lett s a másik: te van, mint akartani rozsdás dalolj nyelve a napból el a szemekkel a re')